In [1]:
import numpy as np
import cv2
import glob
import json
import dlib
from imutils import face_utils
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras_vggface.vggface import VGGFace
import face_recognition
import math

Using TensorFlow backend.


In [2]:
participants = glob.glob('dataset/*')
# participants = sorted(participants)

In [3]:
# Load the dataset
with open('data_cleaned.json') as json_file:
    data_all = json.load(json_file)
# Extract the keys in sorted order
keys_all = sorted(data_all)
# Convert python list to np array
keys_all = np.asarray(keys_all)

In [4]:
data_all.clear()

In [5]:
DATASET_SIZE = len(keys_all)

In [6]:
# detector = dlib.get_frontal_face_detector()
# model = VGGFace(model='resnet50', include_top=False, pooling='avg')

In [11]:
fail_counter = 0
for i in range(50):
    key = keys_all[i]
    
    im = cv2.imread('dataset/' + key)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    label = key.split('/')[1]
    
    # detect the (x, y)-coordinates of the bounding boxes
    # corresponding to each face in the input image
    boxes = face_recognition.face_locations(im, model='hog')
    if len(boxes) == 0:
        print('Failed to detect faces')
        fail_counter += 1
        continue
        
    # Find the correct box assuming that it is the biggest one
    max_dist = 0
    for candidate_box in boxes:
        diagonal_dist = math.sqrt(((candidate_box[0] - candidate_box[2])**2 + (candidate_box[1] - candidate_box[3])**2))
        print(diagonal_dist)
        if diagonal_dist >= max_dist:
            max_dist = diagonal_dist
            box = candidate_box
        else:
            continue
    
    # compute the facial embedding for the face
    encodings = face_recognition.face_encodings(im, [box])
    encodings = encodings[0]

315.3696244092002
315.3696244092002
262.33756879257686
315.3696244092002
315.3696244092002
262.33756879257686
315.3696244092002
314.66331212901196
315.3696244092002
315.3696244092002
262.33756879257686
315.3696244092002
315.3696244092002
315.3696244092002
315.3696244092002
315.3696244092002
315.3696244092002
315.3696244092002
315.3696244092002
315.3696244092002
262.33756879257686
315.3696244092002
262.33756879257686
315.3696244092002
262.33756879257686
314.66331212901196
314.66331212901196
262.33756879257686
262.33756879257686
315.3696244092002
315.3696244092002
315.3696244092002
262.33756879257686
314.66331212901196
315.3696244092002
315.3696244092002
262.33756879257686
315.3696244092002
315.3696244092002
262.33756879257686
314.66331212901196
315.3696244092002
315.3696244092002
315.3696244092002
315.3696244092002
315.3696244092002
315.3696244092002
315.3696244092002
377.59502115361636
315.3696244092002


In [33]:
# Loop over each participant
fail_counter = 0
for j in range(len(participants)):
    
    # Extract the UUID
    uuid_excluded = participants[j].split('/')[1]

    # Loop over the dataset to remove the examples associated with this participant
    indices_excluded = []
    keys_excluded = []
    for i in range(DATASET_SIZE):
        key = keys_all[i]
        uuid = key.split('/')[0]
        if(uuid == uuid_excluded):
            indices_excluded.append(i)
            keys_excluded.append(key)
    
    # Construct the validation dataset consisting of examples of a single participant
    # Evaluation of the classifiers will be done on this dataset
    encodings = np.zeros((len(keys_excluded), 128))
    labels = np.zeros(len(keys_excluded))
    
    failed_indices = []
    for i in range(len(keys_excluded)):
        key = keys_excluded[i]

        im = cv2.imread('dataset/' + key)
        im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        label = key.split('/')[1]

        # detect the (x, y)-coordinates of the bounding boxes
        # corresponding to each face in the input image
        boxes = face_recognition.face_locations(im, model='hog')
        if len(boxes) == 0:
            print('Failed to detect faces')
            failed_indices.append(i)
            fail_counter += 1
            continue

        # Find the correct box assuming that it is the biggest one
        max_dist = 0
        for candidate_box in boxes:
            diagonal_dist = math.sqrt(((candidate_box[0] - candidate_box[2])**2 + (candidate_box[1] - candidate_box[3])**2))
            if diagonal_dist >= max_dist:
                max_dist = diagonal_dist
                box = candidate_box
            else:
                continue

        # compute the facial embedding for the face
        encoding = face_recognition.face_encodings(im, [box])[0]
        encodings[i, :] = encoding
    
    encodings = np.delete(encodings, failed_indices, axis=0)
    np.save('encodings_128/encodings_' + uuid_excluded , encodings)
    np.save('encodings_128/labels_' + uuid_excluded, labels)

Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to detect faces
Failed to d

In [34]:
print(fail_counter)

117


In [28]:
xx = (encodings[:2, :])
print(xx)
print(np.delete(xx, 1, axis=0))

[[-0.14729691  0.06774978  0.05804833 -0.02481486  0.01457742 -0.13098848
   0.03185895 -0.09281449  0.13381694 -0.02344121  0.24778102 -0.09780186
  -0.19955719 -0.10136007  0.00547158  0.13508829 -0.15061416 -0.164839
  -0.05453896 -0.13294765  0.11375745 -0.03479579 -0.04182607  0.05965272
  -0.23127517 -0.29358146 -0.07965374 -0.14851795 -0.00695031 -0.09487604
  -0.02793335  0.07085875 -0.15324566 -0.07099595  0.02489062  0.09382787
   0.06891471 -0.02295644  0.19735321  0.00456644 -0.15250543 -0.01342253
   0.07135649  0.3255024   0.21117303 -0.00125834  0.05611008 -0.00782865
   0.11244152 -0.18898995  0.09594682  0.0941897   0.17605476  0.04345643
   0.12720618 -0.15621096 -0.02702726  0.125782   -0.20915642  0.14669873
   0.08338188 -0.09847712  0.01103789 -0.02662557  0.18027766  0.07074405
  -0.12450655 -0.05133456  0.20990448 -0.11180948 -0.01915999  0.02039182
  -0.14018683 -0.19755314 -0.33523273  0.06714955  0.41887984  0.11236567
  -0.24090514  0.01563269 -0.03446953 -0

In [13]:
# # faces = np.zeros((DATASET_SIZE, 224, 224, 3))
# # labels = np.zeros(DATASET_SIZE)
# failed_indices = []

# for i in range(DATASET_SIZE):
#     key = keys_all[i]
#     im = cv2.imread('dataset/' + key)
#     label = key.split('/')[1]
#     uuid = key.split('/')[0]

#     rects = detector(im, 0)
    
#     min_width, min_height = 0, 0
#     for rect in rects:
#         # convert dlib's rectangle to a OpenCV-style bounding box
#         # [i.e., (x, y, w, h)], then draw the face bounding box
#         (x, y, w, h) = face_utils.rect_to_bb(rect)

#         # Get rid of small faces
#         if(w > min_width and h > min_height):
#             min_width, min_height = w, h
#             face = im[y:y+h, x:x+w]

#     if(face.shape[0] == 0 or face.shape[1] == 0):
#         failed_indices.append(i)
#         continue
            
#     # Resize to match VGGFace requirements
#     face = cv2.resize(face, (224, 224))
        
#     faces[i, :] = face

#     if(label=='positive'):
#         labels[i] = 1
#     else:
#         labels[i] = 0

In [22]:
print(len(failed_indices))

20


In [14]:
faces = np.zeros((1))

In [12]:
features = model.predict(faces)
features_eval = model.predict(faces_eval)

In [19]:
print(features_eval.shape)

(33, 2048)


In [15]:
X_train, y_train = features, labels
X_eval, y_eval = features_eval, labels_eval

In [16]:
svm_classifier = svm.SVC(C=1, kernel='rbf', gamma='scale')
svm_classifier.fit(X_train, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [17]:
y_pred_svm = svm_classifier.predict(X_eval)

In [18]:
print('Training set accuracy for SVM:', svm_classifier.score(X_train, y_train))
print('Test set accuracy for SVM: ', metrics.accuracy_score(y_eval, y_pred_svm))

Training set accuracy for SVM: 0.9323364485981308
Test set accuracy for SVM:  0.45454545454545453


In [16]:
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # turn off summarization, line-wrapping
f = open("VGG_features.txt", "w") 
for row in features:
    f.write(np.array2string(row).split('[')[1].split(']')[0] + '\n')
f = open("VGG_labels.txt", "w")
for label in labels:
    f.write(np.array2string(label) + '\n')

In [14]:
# # Array contain UUIDs of all subjects with their multiplicity
# # e.g. for one subject with 50 sample images the array will contain
# # the subjects' UUID 50 times
# uuids = []
# for i in range(DATASET_SIZE):
#     if i in failed_indices:
#         continue
#     key = keys_all[i]
#     uuid = key.split('/')[0]
#     uuids.append(uuid)

# uuid_lengths = {uuid: uuids.count(uuid) for uuid in uuids}
# uuids = sorted(uuid_lengths)

NameError: name 'DATASET_SIZE' is not defined

In [18]:
with open('uuid_lengths.json', 'w') as f:
    json.dump(uuid_lengths, f)

In [16]:
# uuid_lengths is a dict with uuids as keys and the number
# of images associated with this uuid as values
with open('uuid_lengths.json') as json_file:
    uuid_lengths = json.load(json_file)

# Load the VGG features for each example. There are 2048 features
# for each image so features.shape = (DATASET_SIZE, 2048)
features = np.loadtxt('VGG_features.txt')
labels = np.loadtxt('VGG_labels.txt')

uuids = sorted(uuid_lengths)

In [33]:
print(uuid_excluded)

0656f5fe35a54d1589e526a702f578b0


In [35]:
features = np.load('VGG_datasets/features_' + uuid_excluded + '.npy')

In [40]:
print(np.sum(features, axis=1))

[2864.3257 2866.558  2845.8967 2928.418  2863.4175 2834.2725 2992.9329
 2911.8062 2874.623  2562.122  2953.2136 2919.2275 2852.8413 2817.1284
 2948.075  2867.8906 2884.066  2957.975  2914.134  2883.3782 2823.1348
 2937.5815 2893.3374 2973.7566 2942.027  2891.0073 2975.7183 3012.918
 2942.0112 2899.5078 2869.3372 2877.6182 2857.1086]


In [41]:
print(np.sum(features_eval, axis=1))

[2864.3257 2866.558  2845.8967 2928.418  2863.4175 2834.2725 2992.9329
 2911.8062 2874.623  2562.122  2953.2136 2919.2275 2852.8413 2817.1284
 2948.075  2867.8906 2884.066  2957.975  2914.134  2883.3782 2823.1348
 2937.5815 2893.3374 2973.7566 2942.027  2891.0073 2975.7183 3012.918
 2942.0112 2899.5078 2869.3372 2877.6182 2857.1086]


In [27]:
# Extract and save the features for each UUID
start = 0
for i in range(len(uuids)):
    uuid = participants[i].split('/')[1]
    current_features = features[start:start + uuid_lengths[uuid], :]
    current_labels = labels[start:start + uuid_lengths[uuid]]
    np.save('VGG_datasets/features_' + uuids[i], current_features)
    np.save('VGG_datasets/labels_' + uuids[i], current_labels)
    start += uuid_lengths[uuid]

In [28]:
print(uuids)

['04b7b555cd6d4d41bc2ec8ed6ee259e0', '05d77af9b4ce4de5b0f631f685601d41', '05e84e3f8e6344f5baaf003543df8d3a', '0656f5fe35a54d1589e526a702f578b0', '0c69c12c3544409f82fbcc522fb522f8', '0e55dc4aec5e442f86b809cacdec3b68', '0e7047653b014264b33c1c3620e506f8', '149797134bae4deaac692a73bf162fe9', '2cc97c9c49cb4f03ae978d1eb73f304b', '319c7f878ccb4d53948c4b338cc1732a', '36d8e15f7e164b5c980492c4c6019322', '3c243889b1c547418c69b234216cea7d', '3ef8c45b30ec40cb90526280c04dec0a', '46056754fae5429e975f17d143673451', '49102755134149d98bec4d88dec02a3c', '493e717904f744978a20292fb4adddaa', '50b8db359b164716b59c5e3a9fd001f7', '5f85582c5f1b40b48440af9c34ff3249', '61176f72b54241a2aaac74181926195a', '64a48116536441ee926f0430ca699153', '662b0d4736bf4e60858cd03cd107af90', '6ae20e8b86424bc4a624abb83aa2c9f7', '6b89f5b57aad44de9dc40896b024fad4', '744055ae2e9e4d379108bbdbfab4dae5', '7a45e3e8ed4b4774ae98254fd364f2ca', '7c6b6801ba3245bbb10dc6a7b9996d88', '7d9be1d9b22e4a229b71dbf12f339243', '8036631b52854bdda33a928371